## Hope you can help!
Hi there! On behalf of the City of College Station data team, I wanted to thank you for helping us get our data pipelines established.

We currently have a set of sensors between major roads in College Station that monitor traffic statistics. They update every 5 minutes and are available via an API. For our first initiative, we want to begin collecting information on the average speed that people are moving, by cardinal direction. You can see the API call and the pandas code that our analyst has been using to get the data and transform it. 

We were hoping you could help us with each of the following:

1. Move the code below into discrete functions
2. Write the raw results of the API call to an S3 bucket
3. Write the transformed results to a SQLite database. Maybe write it to an Amazon database if you have some extra time. The columns that we would like the table to have are `direction` (str), `speed` (int), and `timestamp` (datetime).
4. Create a deployment and schedule it to run every 5 minutes
5. Setup Prefect Cloud to interact with the pipeline
6. Launch an agent locally to execute flow runs



In [2]:
from sodapy import Socrata
import pandas as pd

In [5]:
# Get our data from https://dev.socrata.com/foundry/data.cstx.gov/g7k6-2zz6
key = "g7k6-2zz6"
client = Socrata("data.cstx.gov", None)
results = client.get(key, limit=200)
print(results[:1])

[{'systemid': 'Iteris', 'origreaderid': 'Texas_Holleman', 'origreaderid_location': {'latitude': '30.61279', 'longitude': '-96.31609', 'human_address': '{"address": "", "city": "", "state": "", "zip": ""}'}, 'destreaderid': 'Texas_Southwest', 'destreaderid_location': {'latitude': '30.60732', 'longitude': '-96.30911', 'human_address': '{"address": "", "city": "", "state": "", "zip": ""}'}, 'origroadway': 'Texas', 'origcrossstreet': 'Holleman', 'origdirection': 'Southbound', 'destroadway': 'Texas', 'destcrossstreet': 'SouthWest', 'destdirection': 'Southbound', 'segmentlenmiles': '0.56', 'datetimerecorded': '2022-05-02T11:55:26.000', 'traveltime': '73', 'speedmph': '28', 'speedmphstddev': '10.41', 'summaryminutes': '15', 'summarysamples': '26', 'mapdisplay': 'True', 'substitutespeed': '-1'}]


In [9]:
results_df = pd.DataFrame.from_records(results)
results_df.head()


,systemid,origreaderid,origreaderid_location,destreaderid,destreaderid_location,origroadway,origcrossstreet,origdirection,destroadway,destcrossstreet,destdirection,segmentlenmiles,datetimerecorded,traveltime,speedmph,speedmphstddev,summaryminutes,summarysamples,mapdisplay,substitutespeed
0,Iteris,Texas_Holleman,"{'latitude': '30.61279', 'longitude': '-96.316...",Texas_Southwest,"{'latitude': '30.60732', 'longitude': '-96.309...",Texas,Holleman,Southbound,Texas,SouthWest,Southbound,0.56,2022-05-02T11:55:26.000,73,28,10.41,15,26,True,-1
1,Iteris,Harvey_Dartmouth,"{'latitude': '30.622261', 'longitude': '-96.31...",Harvey_PostOakMallWest,"{'latitude': '30.625739', 'longitude': '-96.30...",Harvey,Dartmouth,Eastbound,Harvey,PostOakMallWest,Eastbound,0.33,2022-05-02T11:55:26.000,40,30,6.42,15,5,True,-1
2,Iteris,Holleman_Dartmouth,"{'latitude': '30.618541', 'longitude': '-96.30...",Southwest_Dartmouth,"{'latitude': '30.611963', 'longitude': '-96.30...",Dartmouth,Holleman,Southbound,Dartmouth,SouthWest,Southbound,0.5,2022-05-02T11:55:26.000,78,23,1.41,15,2,True,-1
3,Iteris,University_Springloop,"{'latitude': '30.63686', 'longitude': '-96.319...",University_Tarrow,"{'latitude': '30.63311', 'longitude': '-96.329...",University,Spring Loop,Westbound,University,Tarrow,Westbound,0.6,2022-05-02T11:55:26.000,77,28,9.83,15,108,True,-1
4,Iteris,Barron_Victoria,"{'latitude': '30.56039', 'longitude': '-96.287...",WDFitch_Barron,"{'latitude': '30.55498', 'longitude': '-96.293...",Barron,Victoria,Eastbound,Barron,WD Fitch,Eastbound,0.5,2022-05-02T11:55:26.000,-1,-1,-1,15,0,True,-1


In [8]:
# Aggregate data by cardinal direction
results_df["speedmph"] = results_df["speedmph"].astype(int)
speed_by_dir = results_df.groupby("origdirection").mean().speedmph
speed_by_dir

origdirection
Eastbound     22.166667
Northbound    22.150000
Southbound    20.450000
Westbound     19.166667
Name: speedmph, dtype: float64